<a href="https://colab.research.google.com/github/Gullu26/Data-Science-ML/blob/master/forvia_autogen_azure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!source /content/drive/MyDrive/colab_env/bin/activate

In [ ]:
# import sys
# sys.path.append("/content/drive/MyDrive/colab_env/lib/python3.10/site-packages")

# !source /content/drive/MyDrive/colab_env/bin/activate

In [ ]:
# pip install llama_index llama_index.agent.openai langchain langchain_community pyautogen llama_index.llms.azure_openai llama_index.embeddings.huggingface llama_index.llms.huggingface langchain_openai llama-index-llms-langchain


In [ ]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, ServiceContext, load_index_from_storage, StorageContext
from llama_index.core.agent import AgentRunner, ReActAgent,FunctionCallingAgentWorker
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.agent.openai import OpenAIAgentWorker, OpenAIAgent
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from langchain_community.document_loaders import PyMuPDFLoader
from transformers import AutoModelForCausalLM, AutoTokenizer
from llama_index.core.node_parser import SentenceSplitter
from langchain.text_splitter import CharacterTextSplitter
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.agent.openai import OpenAIAgentWorker
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.core import Settings
from autogen import AssistantAgent
import logging
import autogen
import torch
import sys
import os

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [ ]:
class CustomAssistantAgent(AssistantAgent):
    def __init__(self, tools, llm, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.tools = tools
        self.query_engine_tool = next((tool for tool in tools if isinstance(tool, QueryEngineTool)), None)
        self.llm = llm

    def retrieve_relevant_information(self, query):
      if self.query_engine_tool:
        return self.query_engine_tool(query)
      else:
        return "No query engine tool available."

    def generate_response(self, query):
      relevant_information = self.retrieve_relevant_information(query)
      prompt = f"{self.system_message}\n\nHuman: {query}\n\nAssistant: {relevant_information}\n\nHuman: Based on the provided information, please summarize or respond to the query."
      response = self.llm.generate(prompt, max_new_tokens=500, stop="\nHuman:")
      return response.strip()

In [ ]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)

In [ ]:
OPENAI_API_KEY="ad468b0125824c3cbc611ef7753e9642"
AZURE_OPENAI_ENDPOINT="https://abcgenaidemo.openai.azure.com/"
OPENAI_API_VERSION="2023-09-15-preview"
AZURE_OPENAI_DEPLOYMENT="GPT35TURBO"

In [ ]:
llm = AzureOpenAI(
    deployment_name=AZURE_OPENAI_DEPLOYMENT,
    api_key=OPENAI_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=OPENAI_API_VERSION,
)

In [ ]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain_openai import ChatOpenAI, OpenAI
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import HumanMessage

import os
llm = AzureChatOpenAI(
    deployment_name=AZURE_OPENAI_DEPLOYMENT,
    api_key=OPENAI_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=OPENAI_API_VERSION,
)

message = HumanMessage(
    content="Translate this sentence from English to French. I love programming."
)
print(llm.invoke([message]))

def prompt_llm(prompt=None):
    prompt = prompt.replace('\n', ' ').replace('"', "'").replace('\\\\', ' ')

    assert prompt is not None, "Enter prompt with at least 5 tokens"
    assert len(prompt) >= 5, "Enter prompt with at least 5 tokens"

    message_text = [{"role": "system", "content": "Assistant is a large language model trained by OpenAI who gives output in JSON format with keys resolution and assistance number."},
                    {"role": "assistant", "content": f"{prompt}"}]

    completion = openai.ChatCompletion.create(engine=azure_deployment,
                                              messages=message_text,
                                              temperature=0,
                                              max_tokens=800,
                                              top_p=0.95,
                                              frequency_penalty=0,
                                              presence_penalty=0,
                                              stop=None)

    return completion["choices"][0].message

with open('/content/drive/MyDrive/sample_.json') as f:
    data = json.load(f)

prompt = """Based on this JSON object, describe how many rows of tabular data are there?"""

llm_response = prompt_llm(prompt)
print(llm_response['content'])

content="J'adore programmer." response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 20, 'total_tokens': 26}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}} id='run-5d561865-b421-494a-ab36-1ab11c1d5c36-0'


In [ ]:
# model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", torch_dtype=torch.float32, trust_remote_code=True)
# tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)

In [ ]:
# # Define PAD Token = BOS Token
# tokenizer.pad_token = tokenizer.bos_token
# model.config.pad_token_id = model.config.bos_token_id

In [ ]:
# # Create a custom HuggingFaceLLM instance with the phi-2 model and tokenizer
# phi_2_llm = HuggingFaceLLM(model=model, tokenizer=tokenizer, context_window=2048)

# # Use the custom phi_2_llm instance in settings
# Settings.llm = phi_2_llm
# Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
# Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)

In [ ]:
# # Create a VectorStoreIndex
# documents = SimpleDirectoryReader(input_files=["/content/drive/MyDrive/Colab Notebooks/forvia_sample.pdf"]).load_data()
# index = VectorStoreIndex.from_documents(documents)

In [ ]:
# # Create a QueryEngineTool
# query_engine = index.as_query_engine(llm=Settings.llm)
# query_engine_tool = QueryEngineTool.from_defaults(
#     query_engine=query_engine,
#     description="Useful for retrieving specific context from the pdf.",
# )

In [ ]:
# # Define the agents
# bob = CustomAssistantAgent(
#     name="Bob",
#     system_message="You are an expert at analyzing data from the PDF.Say 'TERMINATE' when you are done.",
#     tools=[query_engine_tool],
#     llm=phi_2_llm
# )

# alice = autogen.AssistantAgent(
#     name="Alice",
#     system_message="Provide feedback on Bob's summary.",
# )

# def termination_message(msg):
#     return "TERMINATE" in str(msg.get("content", ""))

# user_proxy = autogen.UserProxyAgent(
#     name="user_proxy",
#     code_execution_config={"use_docker": False},
#     is_termination_msg=termination_message,
#     human_input_mode="NEVER"
# )

# # Create the group chat
# groupchat = autogen.GroupChat(
#     agents=[bob, alice, user_proxy],
#     messages=[]
# )

# # Create the group chat manager
# manager = autogen.GroupChatManager(
#     groupchat=groupchat,
#     code_execution_config={"use_docker": False},
#     is_termination_msg=termination_message
# )

# # Initiate the chat
# user_proxy.initiate_chat(manager, message="Provide a summary of the Pet Survey data from the PDF.")


In [ ]:
# # Define the agents
# bob = CustomAssistantAgent(
#     name="Bob",
#     system_message="You are an expert at analyzing data from the PDF.Say 'TERMINATE' when you are done.",
#     tools=[query_engine_tool],
#     llm=phi_2_llm
# )
# alice = CustomAssistantAgent(
#     name="Alice",
#     system_message="Provide feedback on Bob's summary.",
#     tools=[],
#     llm=phi_2_llm
# )

# # Run the agents
# query = "Provide a summary of the Pet Survey data from the PDF."
# bob_response = bob.generate_response(query)
# print(f"Bob's response: {bob_response}")

# alice_response = alice.generate_response(bob_response)
# print(f"Alice's response: {alice_response}")

In [ ]:
# import torch
# from transformers import AutoModelForCausalLM, AutoTokenizer

# # # Load the Phi-2 model and tokenizer
# #model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", torch_dtype="auto", trust_remote_code=True)
# model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", torch_dtype=torch.float32, trust_remote_code=True)
# tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)

# # Define PAD Token = BOS Token
# tokenizer.pad_token = tokenizer.bos_token
# model.config.pad_token_id = model.config.bos_token_id

In [ ]:
# # Define a single prompt
# prompt = "What is the capital of India?"

# # Tokenize the prompt
# input_ids = tokenizer.encode(prompt, return_tensors="pt")

# # Create the attention mask
# attention_mask = torch.ones_like(input_ids)

# # Generate text using the model
# output = model.generate(
#     input_ids,
#     attention_mask=attention_mask,
#     max_length=100,  # Set a fixed max_length
#     do_sample=True,
#     top_k=50,
#     top_p=0.95,
#     num_return_sequences=1,
#     temperature=0.7,
# )

# # Decode the output
# generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
# print("Prompt:", prompt)
# print("Generated Text:", generated_text)

In [ ]:
### gpt

In [ ]:
Settings.llm = llm
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)

class CustomAssistantAgent(AssistantAgent):
    def __init__(self, tools, llm, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.tools = tools
        self.query_engine_tool = next((tool for tool in tools if isinstance(tool, QueryEngineTool)), None)
        self.llm = llm

    def retrieve_relevant_information(self, query):
      if self.query_engine_tool:
        return self.query_engine_tool(query)
      else:
        return "No query engine tool available."

    def generate_response(self, query):
      relevant_information = self.retrieve_relevant_information(query)
      prompt = f"{self.system_message}\\n\\nHuman: {query}\\n\\nAssistant: {relevant_information}\\n\\nHuman: Based on the provided information, please summarize or respond to the query."
      response = self.llm.generate(prompt, max_new_tokens=500, stop="\\nHuman:")
      return response.strip()

documents = SimpleDirectoryReader(input_files=["/content/drive/MyDrive/Colab Notebooks/forvia_sample.pdf"]).load_data()
index = VectorStoreIndex.from_documents(documents)

query_engine = index.as_query_engine(llm=Settings.llm)
query_engine_tool = QueryEngineTool.from_defaults(
    query_engine=query_engine,
    description="Useful for retrieving specific context from the pdf.",
)

bob = CustomAssistantAgent(
    name="Bob",
    system_message="You are an expert at analyzing data from the PDF.Say 'TERMINATE' when you are done.",
    tools=[query_engine_tool],
    llm=llm
)

alice = autogen.AssistantAgent(
    name="Alice",
    system_message="Provide feedback on Bob's summary.",
)

def termination_message(msg):
    return "TERMINATE" in str(msg.get("content", ""))

user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    code_execution_config={"use_docker": False},
    is_termination_msg=termination_message,
    human_input_mode="NEVER"
)

groupchat = autogen.GroupChat(
    agents=[bob, alice, user_proxy],
    messages=[]
)

manager = autogen.GroupChatManager(
    groupchat=groupchat,
    code_execution_config={"use_docker": False},
    is_termination_msg=termination_message
)

user_proxy.initiate_chat(manager, message="Provide a summary of the Pet Survey data from the PDF.")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

user_proxy (to chat_manager):

Provide a summary of the Pet Survey data from the PDF.

--------------------------------------------------------------------------------
Bob (to chat_manager):



--------------------------------------------------------------------------------
Alice (to chat_manager):



--------------------------------------------------------------------------------
user_proxy (to chat_manager):



--------------------------------------------------------------------------------
Bob (to chat_manager):



--------------------------------------------------------------------------------
Alice (to chat_manager):



--------------------------------------------------------------------------------
user_proxy (to chat_manager):



--------------------------------------------------------------------------------
Bob (to chat_manager):



--------------------------------------------------------------------------------
Alice (to chat_manager):



-------------------------------------

ChatResult(chat_id=None, chat_history=[{'content': 'Provide a summary of the Pet Survey data from the PDF.', 'role': 'assistant'}, {'content': '', 'name': 'Bob', 'role': 'user'}, {'content': '', 'name': 'Alice', 'role': 'user'}, {'content': '', 'role': 'assistant'}, {'content': '', 'name': 'Bob', 'role': 'user'}, {'content': '', 'name': 'Alice', 'role': 'user'}, {'content': '', 'role': 'assistant'}, {'content': '', 'name': 'Bob', 'role': 'user'}, {'content': '', 'name': 'Alice', 'role': 'user'}, {'content': '', 'role': 'assistant'}], summary='', cost={'usage_including_cached_inference': {'total_cost': 0}, 'usage_excluding_cached_inference': {'total_cost': 0}}, human_input=[])

In [ ]:
class CustomAssistantAgent(autogen.AssistantAgent):
    def __init__(self, tools, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.tools = tools

agent = CustomAssistantAgent(
    name="PDF Summarizer",
    system_message="You are an expert at analyzing data from PDFs. Use the QueryEngineTool to retrieve relevant information from the PDF and provide a summary or response based on the query.",
    tools=[query_engine_tool],
    #llm=llm,
)

user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    code_execution_config={"use_docker": False},
    human_input_mode="NEVER"
)

groupchat = autogen.GroupChat(
    agents=[agent, user_proxy],
    messages=[],
    speaker_selection_method='round_robin'
)

manager = autogen.GroupChatManager(
    groupchat=groupchat,
    code_execution_config={"use_docker": False}
)

user_proxy.initiate_chat(manager, message="Provide a summary of the Pet Survey data from the PDF.")



user_proxy (to chat_manager):

Provide a summary of the Pet Survey data from the PDF.

--------------------------------------------------------------------------------
PDF Summarizer (to chat_manager):



--------------------------------------------------------------------------------
user_proxy (to chat_manager):



--------------------------------------------------------------------------------
PDF Summarizer (to chat_manager):



--------------------------------------------------------------------------------
user_proxy (to chat_manager):



--------------------------------------------------------------------------------
PDF Summarizer (to chat_manager):



--------------------------------------------------------------------------------
user_proxy (to chat_manager):



--------------------------------------------------------------------------------
PDF Summarizer (to chat_manager):



--------------------------------------------------------------------------------
user_proxy (to cha

ChatResult(chat_id=None, chat_history=[{'content': 'Provide a summary of the Pet Survey data from the PDF.', 'role': 'assistant'}, {'content': '', 'name': 'PDF Summarizer', 'role': 'user'}, {'content': '', 'role': 'assistant'}, {'content': '', 'name': 'PDF Summarizer', 'role': 'user'}, {'content': '', 'role': 'assistant'}, {'content': '', 'name': 'PDF Summarizer', 'role': 'user'}, {'content': '', 'role': 'assistant'}, {'content': '', 'name': 'PDF Summarizer', 'role': 'user'}, {'content': '', 'role': 'assistant'}, {'content': '', 'name': 'PDF Summarizer', 'role': 'user'}], summary='', cost={'usage_including_cached_inference': {'total_cost': 0}, 'usage_excluding_cached_inference': {'total_cost': 0}}, human_input=[])

In [ ]:
class CustomAssistantAgent(autogen.AssistantAgent):
    def __init__(self, tools, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.tools = tools

agent = CustomAssistantAgent(
    name="PDF Summarizer",
    system_message="You are an expert at analyzing data from PDFs. Use the QueryEngineTool to retrieve relevant information from the PDF and provide a summary or response based on the query.",
    tools=[query_engine_tool],
    #llm=llm,
)

query = "Provide a summary of the Pet Survey data from the PDF."
response = agent.run(query)
print(response)

AttributeError: 'CustomAssistantAgent' object has no attribute 'run'